<a href="https://colab.research.google.com/github/mojtaba732/ML_Practice/blob/main/decision_tree_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from collections import Counter

In [ ]:
def entropy(x):
  h = np.bincount(x)
  ps = h / len(x)
  return -np.sum([p*np.log2(p) for p in ps if p>0])

In [ ]:
class Node:
  def __init__(self,feature = None, threshold = None, left = None,
                              right = None, *,value = None ):
    self.feature = feature
    self.threshold = threshold
    self.left = left
    self.right = right
    self.value = value

  def is_leaf_node(self):
    return self.value is not None

In [ ]:
class DecisionTree:
  def __init__(self,min_sample_split = 2, max_depth = 100, n_features = None):
    self.min_sample_split = min_sample_split
    self.max_depth = max_depth
    self.n_features = n_features
    self.root = None

  def fit(self,X,y):
    #Grow tree
    self.n_features = X.shape[1] if not self.n_features else min(self.n_features,X.shape[1])
    self.root = self._grow_tree(X,y)


  def _grow_tree(self,X,y,depth = 0):
    n_samples , n_feats = X.shape
    n_labels = len(np.unique(y))

    #stoping criteria
    if (depth >= self.max_depth
        or n_labels ==1
        or n_samples < self.min_sample_split):
      leaf_value = self._most_common_label(y)
      return Node(value = leaf_value)

    feature_idxs = np.random.choice(n_feats, self.n_features,replace=False)
    #Greedy search
    best_feature, best_threshold = self._best_criteria(X,y,feature_idxs)
    left_idxs , right_idxs = self._split(X[:,best_feature],best_threshold)
    left = self._grow_tree(X[left_idxs,:],y[left_idxs],depth+1)
    right = self._grow_tree(X[right_idxs,:],y[right_idxs],depth+1)
    return Node(best_feature, best_threshold, left, right)


  def _most_common_label(self,y):
    counter =  Counter(y)
    most_common = counter.most_common(1)[0][0]
    return most_common

  def _best_criteria(self,X,y,feature_idxs):
    best_gain = -1
    split_idx, split_thresh = None,None
    for f_idx in feature_idxs:
      X_column = X[:,f_idx]
      thresholds = np.unique(X_column)
      for threshold in thresholds:
        gain = self._information_gain(y,X_column,threshold)
        if gain > best_gain:
          best_gain = gain
          split_thresh = threshold
          split_idx = f_idx

    return split_idx,split_thresh

  def _information_gain(self,y,X_column,split_thresh):
    #Parent Entroy
    parent_entropy = entropy(y)

    #Generate split
    left_idx, right_idx = self._split(X_column,split_thresh)
    if len(left_idx) == 0 or len(right_idx)==0 :
      return 0
    #Calculate IG
    n = len(y)
    n_l , n_r = len(left_idx) , len(right_idx)
    e_l , e_r = entropy(y[left_idx]) , entropy(y[right_idx])
    child_entropy = (n_l/n)*e_l+(n_r/n)*e_r
    ig =parent_entropy - child_entropy
    return ig


  def _split(self, X_column,split_thresh):
    left_idx = np.argwhere(X_column <= split_thresh).flatten()
    right_idx = np.argwhere(X_column > split_thresh).flatten()
    return left_idx,right_idx




  def predict(self,X):
    #Traverse tree
    return np.array([self._traverse_tree(x,self.root) for x in X])

  def _traverse_tree(self,x,node):
    if node.is_leaf_node():
      return node.value

    if x[node.feature] <= node.threshold :
      return self._traverse_tree(x,node.left)
    return self._traverse_tree(x,node.right)




In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split

def accuracy(y_true,y_pred):
  accuracy = np.sum(y_true == y_pred)/len(y_true)
  return accuracy


data =  datasets.load_breast_cancer()
X = data.data
y = data.target


X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=5)
clf = DecisionTree(max_depth=10)
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
acc = accuracy(y_test,y_pred)
print(f"The Accuracy is {acc}")

The Accuracy is 0.9473684210526315
